In [2]:
from pathlib import Path
import numpy as np
import tifffile
from skimage.exposure import match_histograms, rescale_intensity
from skimage.restoration import calibrate_denoiser, denoise_tv_chambolle
import matplotlib.pyplot as plt
from tqdm import tqdm
import os


def optical_sectioning_sim_90deg(imgs, method):
    # separate 0, 90, 180 images
    I1 = imgs[0,:]
    I2 = imgs[1,:]
    I3 = imgs[2,:]

    # calculate OS image using law of sines to recover in-focus intensities
    if method == 'DD':
        os_image = 0.5 * np.sqrt((2*I2 - I1 - I3)**2 + (I3 - I1)**2) 

    elif method == 'NEIL':
        os_image = np.sqrt(((I1-I2)**2)+((I1-I3)**2)+((I2-I3)**2))

    else:
        print('Invalid method. Please choose either DD or NEIL')  
    
    return os_image

def match_histogram_z(imgs, nangles, nphases):
    for ii in range(nangles):
        for jj in range(1, nphases):
            imgs[ii, jj,:] = match_histograms(imgs[ii, jj,:], imgs[ii, 0,:])

    return imgs

def run_SIM(file_Name, method):
    dx = 1.0
    dz = 1.0
    excitation_wl = 0.470
    emission_wl = 0.520
    na = 0.3
    nangles = 1
    nphases = 3

    # load the data and reshape
    input_file_path = Path(file_Name)
    #input_file_path = Path("c://users//researcher/downloads/TRYSIM_8.tif")
    root_path = input_file_path.parents[0]
    img = tifffile.imread(input_file_path)

    # TO DO: correctly parse metadata / load multiple images loop over timelapse
    nt = 1
    nz = int(img.shape[0]/(nangles*nphases))
    ny = img.shape[1]
    nx = img.shape[2]
    img_reshape = np.reshape(img,[nz,nangles,nphases,ny,nx])

    # turn image into float
    img_reshape = img_reshape.astype(np.float32)

    # create storage variables
    widefield = np.zeros((nz,ny,nx),dtype=np.float32)
    os_sim_per_angle = np.zeros((nangles,ny,nx),dtype=np.float32)
    os_sim_angle = np.zeros((nangles,nz,ny,nx),dtype=np.float32)
    os_sim = np.zeros((nz,ny,nx),dtype=np.float32)


    # loop over all timepoints
    for t_idx in tqdm(range(0,nt),desc='time',leave=True):

        # check if there is more than one time point
        if nt==1:
            imgs_to_process = img_reshape[:]
        else:
            imgs_to_process = img_reshape[t_idx,:]
        
        for z_idx in tqdm(range(0,nz),desc='SIM OS per z plane',leave=False):
            
            # match histograms for each phase & angle at this z plane
            matched_imgs = match_histogram_z(imgs_to_process[z_idx,:],nangles,nphases)

            # calculate widefield image at this z plane
            widefield[z_idx,:] = np.nanmean(matched_imgs, axis=(0, 1))
            
            # calculate os-sim image for each angle at this z plane
            for angle_idx in range(0,nangles):
                os_sim_per_angle[angle_idx,:]=optical_sectioning_sim_90deg(matched_imgs[angle_idx,:],method)
                os_sim_angle[angle_idx,z_idx,:]=os_sim_per_angle[angle_idx,:]
            # average os-sim over all angles at this z plane
            os_sim[z_idx,:] = np.nanmean(os_sim_per_angle,axis=0)


        # rescale to 16-bit for output
        widefield = rescale_intensity(widefield,out_range=(0,65535)).astype(np.uint16)
        os_sim = rescale_intensity(os_sim,out_range=(0,65535)).astype(np.uint16)

        return widefield, os_sim
    


In [3]:
def save_Reconstructions(widefield,os_sim, input_file):
    output_file_path_SIM = Path(input_file.rsplit('.', 1)[0] + '_SIM_Reconstruction.tif')
    output_file_path_pWF = Path(input_file.rsplit('.', 1)[0] + '_pWF_Reconstruction.tif')
    tifffile.imwrite(output_file_path_SIM,os_sim)
    tifffile.imwrite(output_file_path_pWF,widefield)


In [4]:
def process_Multiple_Files(folder_Name, method):
    directory = os.fsencode(folder_Name)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".tif"):
            
            print(file)
            widefield, os_sim = run_SIM(folder_Name + '\\' + filename, method)
            save_Reconstructions(widefield,os_sim, folder_Name +'\\'+ filename)
            continue
        else:
            continue
        

In [5]:
process_Multiple_Files('GCaMP6_053123','DD')

b'200Hz_TIC.tif'


time:   0%|          | 0/1 [00:12<?, ?it/s]


b'200Hz_With_TI.tif'


time:   0%|          | 0/1 [02:23<?, ?it/s]


b'SIMSIM_GCAMP1170.tif'


...

time:   0%|          | 0/1 [00:06<?, ?it/s]


b'SIMSIM_GC_DD40_1.tif'


time:   0%|          | 0/1 [00:06<?, ?it/s]


b'try1_1666.tif'


time:   0%|          | 0/1 [00:00<?, ?it/s]
